# UAT-C3: The Training Engine
This notebook verifies the `PacemakerTrainer` module.

In [ ]:
import subprocess
import sys
import tempfile
from pathlib import Path
from unittest.mock import patch

from ase.build import bulk
from ase.db import connect

sys.path.append(str(Path.cwd().parent.parent))

from mlip_autopipec.config.training import TrainingConfig
from mlip_autopipec.modules.training import PacemakerTrainer

### Step 1: Prepare Test Environment

In [ ]:
temp_dir = tempfile.TemporaryDirectory()
work_dir = Path(temp_dir.name)
db_path = work_dir / "test_data.db"

with connect(db_path) as db:
    atoms = bulk("Si", "diamond", a=5.43)
    atoms.info["energy"] = -15.0
    atoms.arrays["forces"] = atoms.get_positions() * 0.0
    db.write(atoms)

### Step 2: Configure and Run Trainer (Mocked)

In [ ]:
template_file_path = work_dir / "pacemaker.in.j2"
template_file_path.write_text('key = value')

training_config = TrainingConfig(
    pacemaker_executable=Path("/bin/true"),
    data_source_db=db_path,
    template_file=template_file_path
)

trainer = PacemakerTrainer(training_config=training_config)

with patch("subprocess.run") as mock_run:
    mock_run.return_value = subprocess.CompletedProcess(
        args=[], returncode=0, stdout="Final potential saved to: potential.yace", stderr=""
    )

    def create_dummy_potential(*args, **kwargs):
        (kwargs.get("cwd") / "potential.yace").touch()
        return mock_run.return_value

    mock_run.side_effect = create_dummy_potential
    potential_path = trainer.train()

assert potential_path.exists()
assert potential_path.name == "potential.yace"

In [ ]:
temp_dir.cleanup()